Author: Xinnan Shen<br>
Date: 05-05-2020

# Learned Index Model Simple Demo
This notebook is to build some simple learned index models based on Kraska's paper.

Step 1: Generate some simple dataset in csv format

In [1]:
import os
import codecs
import random
#function: data_generation
#usage: generate a simple dataset
#parameters:
#1.len_num: the size of dataset
#2.range_min: the minimum key
#3.range_max: the maximun key
#output dataset: two columns (key,location)
def data_generation(len_num,range_min,range_max):
	datalist=[]
	for i in range(0,len_num):
		x=random.randint(range_min,range_max)
		datalist.append(x)
	for i in range(0,len(datalist)):
		temp=False
		for j in range(0,len(datalist)-i-1):
			if datalist[j]>datalist[j+1]:
				t=datalist[j]
				datalist[j]=datalist[j+1]
				datalist[j+1]=t
				temp=True
		if not temp:
			break
	current_path=os.path.abspath(os.curdir)
	f=codecs.open(os.path.join(current_path,"data.csv"), "w", "utf-8")
	for i in range(0,len(datalist)):
		f.write(str(datalist[i])+","+str(i)+"\n")
	f.close()
	return

Provide some value and generate the dataset

In [2]:
minkey=1000
maxkey=9999
keynum=3000
data_generation(3000,1000,9999)

Step 2: Split the dataset into training, development and testing dataset

In [3]:
from random import shuffle
import numpy as np
from sklearn.model_selection import train_test_split
current_path=os.path.abspath(os.curdir)
f=codecs.open(os.path.join(current_path,"data.csv"), "r", "utf-8")
strlist=f.read().split("\n")
f.close()
list_key=[]
list_res=[]
for ele in strlist:
    temp=ele.split(",")
    if len(temp)!=2:
        continue
    list_key.append(temp[0])
    list_res.append(temp[1])
keys=np.array(list_key)
res=np.array(list_res)
trainkeys,testkeys,trainres,testres=train_test_split(keys,res,test_size=0.35)
trainkeys,devkeys,trainres,devres=train_test_split(trainkeys,trainres,test_size=0.5)
trainkeys=list(trainkeys)
devkeys=list(devkeys)
testkeys=list(testkeys)
trainres=list(trainres)
devres=list(devres)
testres=list(testres)

f=codecs.open(os.path.join(current_path,"data_train.csv"), "w", "utf-8")
for i in range(0,len(trainkeys)):
    f.write(str(trainkeys[i])+","+str(trainres[i])+"\n")
f.close()
f=codecs.open(os.path.join(current_path,"data_dev.csv"), "w", "utf-8")
for i in range(0,len(devkeys)):
    f.write(str(devkeys[i])+","+str(devres[i])+"\n")
f.close()
f=codecs.open(os.path.join(current_path,"data_test.csv"), "w", "utf-8")
for i in range(0,len(testkeys)):
    f.write(str(testkeys[i])+","+str(testres[i])+"\n")
f.close()
print("training data size:",len(trainkeys))
print("development data size:",len(devkeys))
print("testing data size:",len(testkeys))

training data size: 975
development data size: 975
testing data size: 1050


In [4]:
import codecs
import os
minkey=1000
maxkey=9999
keynum=3000
current_path=os.path.abspath(os.curdir)
f=codecs.open(os.path.join(current_path,"data_train.csv"), "r", "utf-8")
strlist=f.read().split("\n")
f.close()
trainkeys=[]
trainres=[]
for ele in strlist:
    temp=ele.split(",")
    if len(temp)!=2:
        continue
    trainkeys.append(int(temp[0]))
    trainres.append(int(temp[1]))
f=codecs.open(os.path.join(current_path,"data_dev.csv"), "r", "utf-8")
strlist=f.read().split("\n")
f.close()
devkeys=[]
devres=[]
for ele in strlist:
    temp=ele.split(",")
    if len(temp)!=2:
        continue
    devkeys.append(int(temp[0]))
    devres.append(int(temp[1]))
f=codecs.open(os.path.join(current_path,"data_test.csv"), "r", "utf-8")
strlist=f.read().split("\n")
f.close()
testkeys=[]
testres=[]
for ele in strlist:
    temp=ele.split(",")
    if len(temp)!=2:
        continue
    testkeys.append(int(temp[0]))
    testres.append(int(temp[1]))
print("training data size:",len(trainkeys))
print("development data size:",len(devkeys))
print("testing data size:",len(testkeys))

training data size: 975
development data size: 975
testing data size: 1050


Step 3: Building a simple B-Tree model

In [9]:
#This is a B-Tree model
#reference: https://www.jianshu.com/p/c625a009e488
from random import shuffle
import random
import os
import codecs
import numpy as np
import math
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error 
mse_BTree=0.0
root_node = None
trainkeys=[]
trainres=[]
devkeys=[]
devres=[]
testkeys=[]
testres=[]
class Logger(object):
    @classmethod
    def tree(cls, node, child_name, dsc, depth):
        if depth == 0:
            head = "|   " * depth
            print(head + "+--" + dsc(node))
            depth = depth + 1
        for child in getattr(node, child_name):
            head = "|   " * depth
            print(head + "+--" + dsc(child))
            cls.tree(child, child_name, dsc, depth + 1)
class BKeyword(object):
    def __init__(self, key, loc):
        self.key = key
        self.loc = loc
class BNode(object):
    def __init__(self,M):
        self._parent: BNode = None
        self.keywords = []
        self.child_nodes = []
        self.M=M
    # set parent node
    def set_parent(self, node):
        self._parent = node
        if node.get_parent() is None:
            global root_node
            root_node = node.get_parent()
    # get parent node
    def get_parent(self):
        return self._parent
    # add child node to right location
    def insert_child_node(self, index, add_node):
        add_node.set_parent(self)
        self.child_nodes.insert(index, add_node)
    # add child node
    def append_child_node(self, add_node):
        add_node.set_parent(self)
        self.child_nodes.append(add_node)
    # find right insertion location
    def find_add_index(self, add_word):
        if len(self.keywords) == 0:
            return 0
        index = 0
        while True:
            if index >= len(self.keywords):
                break
            key = self.keywords[index].key
            if add_word.key < key:
                break
            index = index + 1
        return index
	#find the location of given keyword
    def find_loc(self,word):
        if len(self.keywords) == 0:
            return -1
        index = 0
        key=-1
        while True:
            if index >= len(self.keywords):
                break
            key = self.keywords[index].key
            if word < key:
                break
            index = index + 1
        if index==0:
            index=1
        index=index-1
        #print(index)
        if index+1>=len(self.keywords):
            return int(self.keywords[index].loc)
        if self.keywords[index].key==word or abs(int(word)-int(self.keywords[index].key))<abs(int(word)-int(self.keywords[index+1].key)):
            return int(self.keywords[index].loc)
        else:
            return int(self.keywords[index+1].loc)
    # insert data to right location (regardless of M)
    def blind_add(self, word: BKeyword) -> int:
        index = self.find_add_index(word)
        self.keywords.insert(index, word)
    def split(self):
        # split node
        parent, center_keyword, left_node, right_node = self.split_to_piece()
        # add two new nodes as parent, build relationship
        parent_add_index = parent.find_add_index(center_keyword)
        parent.insert_child_node(parent_add_index, right_node)
        parent.insert_child_node(parent_add_index, left_node)
        # remove itself
        if self in parent.child_nodes:
            parent.child_nodes.remove(self)
        parent.add_word(center_keyword, force=True)
        # redefine root
        root = self
        while root.get_parent() is not None:
            root = root.get_parent()
        global root_node
        root_node = root
    def split_to_piece(self):
        center_keyword = self.keywords[int((self.M-1)/2)]
        if self.get_parent() is None:
            self.set_parent(BNode(self.M))
        left_node = BNode(self.M)
        right_node = BNode(self.M)
        for keyword in self.keywords:
            if keyword.key < center_keyword.key:
                left_node.keywords.append(keyword)
            elif keyword.key > center_keyword.key:
                right_node.keywords.append(keyword)
        for i in range(len(self.child_nodes)):
            if i <= int((len(self.child_nodes) - 1)/2):
                left_node.append_child_node(self.child_nodes[i])
            else:
                right_node.append_child_node(self.child_nodes[i])
        return self.get_parent(), center_keyword, left_node, right_node
    def add_word(self, keyword, force=False):
        if len(self.child_nodes) == 0 or force:
            self.blind_add(keyword)
            if len(self.keywords) == self.M:
                self.split()
        else:

            index = self.find_add_index(keyword)
            if index >= len(self.child_nodes):
                index = index - 1
            self.child_nodes[index].add_word(keyword)
def B_Tree_Model():
    print("Simple B-Tree Model")
    root_node=BNode(4)
    for i in range(0,len(trainkeys)):
        keyword=BKeyword(trainkeys[i],trainres[i])
        root_node.add_word(keyword)
    devpre=[]
    for i in range(0,len(devkeys)):
        devpre.append(root_node.find_loc(devkeys[i]))
    global mse_BTree
    mse_BTree=mean_squared_error(devres,devpre)
    print("log MSE dev: ",round(math.log(1+mse_BTree,2),3))
    testpre=[]
    for i in range(0,len(testkeys)):
        testpre.append(root_node.find_loc(testkeys[i]))
    print("log MSE test: ",round(math.log(1+mean_squared_error(testres,testpre),2),3))
    return
if __name__ == '__main__':
    f=codecs.open(os.path.join(current_path,"data_train.csv"), "r", "utf-8")
    strlist=f.read().split("\n")
    f.close()
    trainkeys=[]
    trainres=[]
    for ele in strlist:
        temp=ele.split(",")
        if len(temp)!=2:
            continue
        trainkeys.append(int(temp[0]))
        trainres.append(int(temp[1]))
    f=codecs.open(os.path.join(current_path,"data_dev.csv"), "r", "utf-8")
    strlist=f.read().split("\n")
    f.close()
    devkeys=[]
    devres=[]
    for ele in strlist:
        temp=ele.split(",")
        if len(temp)!=2:
            continue
        devkeys.append(int(temp[0]))
        devres.append(int(temp[1]))
    f=codecs.open(os.path.join(current_path,"data_test.csv"), "r", "utf-8")
    strlist=f.read().split("\n")
    f.close()
    testkeys=[]
    testres=[]
    for ele in strlist:
        temp=ele.split(",")
        if len(temp)!=2:
            continue
        testkeys.append(int(temp[0]))
        testres.append(int(temp[1]))
    B_Tree_Model()

Simple B-Tree Model
log MSE dev:  2.72
log MSE test:  2.867


Step 4: Build a Linear Regression Model

In [10]:
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.metrics import mean_squared_error 
import math
print("Linear Regression Model")
reg = LinearRegression()
reg.fit(np.array(trainkeys).reshape(-1,1),np.array(trainres).reshape(-1,1))
devpre=reg.predict(np.array(devkeys).reshape(-1,1)).reshape(1,-1).tolist()[0]
for i in range(0,len(devpre)):
    devpre[i]=abs(int(devpre[i]))%keynum
mse_LR=mean_squared_error(devres,devpre)
print("log MSE dev: ",round(math.log(mse_LR,2),3))
testpre=reg.predict(np.array(testkeys).reshape(-1,1)).reshape(1,-1).tolist()[0]
for i in range(0,len(testpre)):
    testpre[i]=abs(int(testpre[i]))%keynum
print("log MSE test: ",round(math.log(1+mean_squared_error(testres,testpre),2),3))

Linear Regression Model
log MSE dev:  15.747
log MSE test:  14.078


Step 5: Build a SVR model

In [ ]:
from sklearn.svm import SVR
print("Support Vector Regression Model")
clf_best=None
hp_best={}
best_mse=9999.99
for kernel_ in ['rbf', 'sigmoid']:
    for gamma_ in [1e-4, 1e-3, 0.01, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
        for C_ in [1, 3, 5, 10, 30, 50, 100, 300, 500, 1000, 3000, 5000, 10000, 30000, 50000]:
            clf = SVR(kernel=kernel_,gamma=gamma_,C=C_)
            clf.fit(np.array(trainkeys).reshape(-1,1),np.array(trainres).reshape(-1,1))
            devpre=clf.predict(np.array(devkeys).reshape(-1,1)).reshape(1,-1).tolist()[0]
            for i in range(0,len(devpre)):
                devpre[i]=abs(int(devpre[i]))%keynum
            mse_SVR=mean_squared_error(devres,devpre)
            print("hyper-parameters: ",{'kernel':kernel_,'gamma':gamma_,'C':C_},"log MSE: ",round(math.log(mse_SVR,2),3))
            if mse_SVR<best_mse:
                best_mse=mse_SVR
                hp_best={'kernel':kernel_,'gamma':gamma_,'C':C_}
                clf_best=clf
print("hyper-parameters of best model:",hp_best,"log MSE: ",round(math.log(best_mse,2),3))

Support Vector Regression Model
hyper-parameters:  {'kernel': 'rbf', 'gamma': 0.0001, 'C': 1} log MSE:  19.467
hyper-parameters:  {'kernel': 'rbf', 'gamma': 0.0001, 'C': 3} log MSE:  19.353


/home/sxn/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sxn/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sxn/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


hyper-parameters:  {'kernel': 'rbf', 'gamma': 0.0001, 'C': 5} log MSE:  19.236
hyper-parameters:  {'kernel': 'rbf', 'gamma': 0.0001, 'C': 10} log MSE:  18.928
hyper-parameters:  {'kernel': 'rbf', 'gamma': 0.0001, 'C': 30} log MSE:  17.482


/home/sxn/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sxn/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sxn/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


hyper-parameters:  {'kernel': 'rbf', 'gamma': 0.0001, 'C': 50} log MSE:  15.694


...

/home/sxn/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


hyper-parameters:  {'kernel': 'rbf', 'gamma': 0.001, 'C': 10000} log MSE:  15.47


/home/sxn/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


hyper-parameters:  {'kernel': 'rbf', 'gamma': 0.001, 'C': 30000} log MSE:  15.479


/home/sxn/.local/lib/python3.6/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
